In this Colab, the goal is to recover the intrinsic parameters of a camera given an observation and correspondances between the observation and the render of the current solution.  Things are kept simple by only inserting a rectangle in the 3d scene, and using it as the source of correspondances during the optimization. The minimization is performed using L-BFGS.

In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

#################################
# Imports the necessary modules #
#################################
from colabtools import adhoc_import
with adhoc_import.Google3():
  from tensorflow_graphics.camera import perspective

In [0]:
#############
# Utilities #
#############


def render_rectangle(vertices, focal, principal_point, image_dimensions):
  image = np.zeros((int(image_dimensions[0]), int(image_dimensions[1]), 3))
  vertices_2d = perspective.project(vertices, focal, principal_point)
  with tf.Session() as sess:
    vertices_2d_ = np.array(sess.run([vertices_2d]))
    top_left_corner = np.squeeze(
        np.maximum(vertices_2d_.min(1), (0, 0)).astype(int))
    bottom_right_corner = np.squeeze(
        np.minimum(
            vertices_2d_.max(1),
            (image_dimensions[1] - 1, image_dimensions[0] - 1)).astype(int))
    for x in range(top_left_corner[0], bottom_right_corner[0] + 1):
      for y in range(top_left_corner[1], bottom_right_corner[1] + 1):
        c1 = float(bottom_right_corner[0] + 1 -
                   x) / float(bottom_right_corner[0] + 1 - top_left_corner[0])
        c2 = float(bottom_right_corner[1] + 1 -
                   y) / float(bottom_right_corner[1] + 1 - top_left_corner[1])
        image[y, x] = (c1, c2, 1)
    return image


def plot_optimization_step(target, image):
  plt.figure(figsize=(20, 10))
  ax = plt.subplot("131")
  ax.set_title("Optimization target")
  _ = ax.imshow(target)
  ax = plt.subplot("132")
  ax.set_title("Current solution")
  _ = ax.imshow(image)
  ax = plt.subplot("133")
  ax.set_title("Difference image")
  _ = ax.imshow(np.abs(target - image))
  plt.show()


def print_errors(focal_error, centre_error):
  print("Err focal %f" % (focal_error,))
  print("Err centre %f" % (centre_error,))

In [0]:
##############################################################
# Constructs the camera matrices and the object living in 3d #
##############################################################
image_width = 400
image_height = 300
image_dimensions = np.array((image_height, image_width), dtype=np.float64)
gt_focal = np.array((image_height, image_height), dtype=np.float64)
gt_principal_point = np.array((image_width, image_height)) / 2.0 + (
    np.random.random(2) - 0.5) * image_width / 3.0

# Sets up the vertices of the 3d object
object_depth = 1000.0
vertices = np.array(((-150.0, -75.0, object_depth), (150.0, 75.0,
                                                     object_depth)))

####################
# Builds the graph #
####################
tf.reset_default_graph()

observed_focal = tf.Variable(gt_focal +
                             (np.random.random(2) - 0.5) * image_width)
observed_principal_point = tf.Variable(
    gt_principal_point + (np.random.random(2) - 0.5) * image_width / 4)

vertices_2d_gt = perspective.project(vertices, gt_focal, gt_principal_point)
vertices_2d_observed = perspective.project(vertices, observed_focal,
                                           observed_principal_point)

with tf.name_scope("loss"):
  loss = tf.nn.l2_loss(vertices_2d_gt - vertices_2d_observed)

learning_rate = 0.1
with tf.name_scope("optimizer"):
  optimizer = tf.contrib.opt.ScipyOptimizerInterface(
      loss,
      var_list=[observed_focal, observed_principal_point],
      options={"maxiter": 2})

with tf.name_scope("initialize_variables"):
  init = tf.initialize_all_variables()

session = tf.Session()
session.run(init)

###################################
# Contruction of the target image #
###################################
target = render_rectangle(vertices, gt_focal, gt_principal_point,
                          image_dimensions)

############################
# Display initial solution #
############################
loss_v, observed_focal_v, observed_principal_point_v = session.run(
    [loss, observed_focal, observed_principal_point])
print "Initial configuration:"
print_errors(
    np.linalg.norm(observed_focal_v - gt_focal),
    np.linalg.norm(observed_principal_point_v - gt_principal_point))
image = render_rectangle(vertices, observed_focal_v, observed_principal_point_v,
                         image_dimensions)
plot_optimization_step(target, image)

################
# Optimization #
################
max_iterations = 20
for it in range(max_iterations):
  optimizer.minimize(session)
  loss_v, observed_focal_v, observed_principal_point_v = session.run(
      [loss, observed_focal, observed_principal_point])
  # Display current solution
  print "Loss at iteration %d" % (it + 1,)
  image = render_rectangle(vertices, observed_focal_v,
                           observed_principal_point_v, image_dimensions)
  print_errors(
      np.linalg.norm(observed_focal_v - gt_focal),
      np.linalg.norm(observed_principal_point_v - gt_principal_point))
  plot_optimization_step(target, image)
  if loss_v < 0.5:
    break